In [1]:
%load_ext autoreload
%autoreload 2
import generator
import featuretools as ft
import pandas as pd
import os
from glob import glob
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('max_colwidth', 3_000)

### Много таблиц

In [3]:
# Параметры для спецификации нескольких таблиц
parameters_many_tables_all_features = {
    'path_data': '/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data',
    'tables': {
        'app.csv': 'SK_ID_CURR',
        'bureau_balance.csv': False,
        'bureau.csv': 'SK_ID_BUREAU',
        'previous.csv': 'SK_ID_PREV',
        'cash.csv': False,
        'installments.csv': False,
        'credit.csv': False
    },  # в качестве значений указываются уникальные индексы для таблиц
    'main_table': 'app.csv',  # у main_table ОБЯЗАТЕЛЬНО должен быть уникальный идентификатор
    'target': 'TARGET',  # таргет в main_table
    'depth': 2,
    'relations': [
        [('app.csv', 'SK_ID_CURR'), ('bureau.csv', 'SK_ID_CURR')],
        [('bureau.csv', 'SK_ID_BUREAU'), ('bureau_balance.csv', 'SK_ID_BUREAU')], 
        [('app.csv','SK_ID_CURR'), ('previous.csv','SK_ID_CURR')], 
        [('previous.csv', 'SK_ID_PREV'), ('cash.csv', 'SK_ID_PREV')], 
        [('previous.csv', 'SK_ID_PREV'), ('installments.csv', 'SK_ID_PREV')], 
        [('previous.csv', 'SK_ID_PREV'), ('credit.csv', 'SK_ID_PREV')],
    ],
    'time_indecies': {
        'app.csv': False,
        'bureau_balance.csv': 'bureau_balance_date',
        'bureau.csv': False,
        'cash.csv': False,
        'credit.csv': 'credit_balance_date',
        'installments.csv': 'installments_paid_date', 
        'previous.csv': False,
    },
    'time_variables': {
        'app.csv': False,
        'bureau_balance.csv': ['bureau_balance_date'],
        'bureau.csv': ['bureau_credit_application_date', 'bureau_credit_end_date', 'bureau_credit_close_date', 'bureau_credit_update_date'],
        'cash.csv': ['cash_balance_date'],
        'credit.csv': ['credit_balance_date'],
        'installments.csv': ['installments_due_date', 'installments_paid_date'], 
        'previous.csv': ['previous_decision_date', 'previous_drawing_date', 'previous_first_due_date', 'previous_last_duefirst_date'],
    },  # после получения от Ани таблицы с типами, удалить этот словарь, в этих списках должны быть поля time_index'
    'sep': ',',  # разделитель в csv файлах
    'chunk_size': 0.5,  # количество строчек обрабатываемых за момент времени (если число от 0 до 1 - то это процент строчек), рекомендуется использовать 1/n_jobs
    'max_features': 10**5,  # максимальное количество генерируемых фич
    'n_jobs': 2,  # количество потоков, в которых запускается генерация фичей
    'agg_primitives': ['count', 'max', 'min', 'mean', 'num_unique', 'percent_true', 'all', 'time_since_last'],  # добавлены примитивы для работы с датами
#     'agg_primitives': ['max'], # 'min'],
#     'trans_primitives': ['percentile'],
    'trans_primitives': ['percentile', 'not', 'cum_sum', 'cum_max', 'cum_mean', 'cum_min', 'month', 'day', 'weekday', 'week', 'time_since_previous'],  # добавлены примитивы для работы с датами
    'output_file_name': 'fm_many_tables_all_features.csv',
    'file_name_for_features': 'fn_many_tables.json',  # имя файла для сохранения списка сгенерированных фич (используется для отбора фич после модуля selector)
    'generate_selected_features': False,  # указание на то, что нужно сгенерировать только выбранные фичи модулем selector
    'file_with_selected_features': None  # результирующая таблица после модуля selector
}

parameters_many_tables_selected_features = {
    'path_data': '/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data',
    'tables': {
        'app.csv': 'SK_ID_CURR',
        'bureau_balance.csv': False,
        'bureau.csv': 'SK_ID_BUREAU',
        'previous.csv': 'SK_ID_PREV',
        'cash.csv': False,
        'installments.csv': False,
        'credit.csv': False
    },  # в качестве значений указываются уникальные индексы для таблиц
    'main_table': 'app.csv',  # у main_table ОБЯЗАТЕЛЬНО должен быть уникальный идентификатор
    'target': 'TARGET',  # таргет в main_table
    'depth': 2,
    'relations': [
        [('app.csv', 'SK_ID_CURR'), ('bureau.csv', 'SK_ID_CURR')],
        [('bureau.csv', 'SK_ID_BUREAU'), ('bureau_balance.csv', 'SK_ID_BUREAU')], 
        [('app.csv','SK_ID_CURR'), ('previous.csv','SK_ID_CURR')], 
        [('previous.csv', 'SK_ID_PREV'), ('cash.csv', 'SK_ID_PREV')], 
        [('previous.csv', 'SK_ID_PREV'), ('installments.csv', 'SK_ID_PREV')], 
        [('previous.csv', 'SK_ID_PREV'), ('credit.csv', 'SK_ID_PREV')],
    ],
    'time_indecies': {
        'app.csv': False,
        'bureau_balance.csv': 'bureau_balance_date',
        'bureau.csv': False,
        'cash.csv': False,
        'credit.csv': 'credit_balance_date',
        'installments.csv': 'installments_paid_date', 
        'previous.csv': False,
    },
    'time_variables': {
        'app.csv': False,
        'bureau_balance.csv': ['bureau_balance_date'],
        'bureau.csv': ['bureau_credit_application_date', 'bureau_credit_end_date', 'bureau_credit_close_date', 'bureau_credit_update_date'],
        'cash.csv': ['cash_balance_date'],
        'credit.csv': ['credit_balance_date'],
        'installments.csv': ['installments_due_date', 'installments_paid_date'], 
        'previous.csv': ['previous_decision_date', 'previous_drawing_date', 'previous_first_due_date', 'previous_last_duefirst_date'],
    },  # после получения от Ани таблицы с типами, удалить этот словарь, в этих списках должны быть поля time_index'
    'sep': ',',  # разделитель в csv файлах
    'chunk_size': 0.5,  # количество строчек обрабатываемых за момент времени (если число от 0 до 1 - то это процент строчек), рекомендуется использовать 1/n_jobs
    'max_features': 10**5,  # максимальное количество генерируемых фич
    'n_jobs': 2,  # количество потоков, в которых запускается генерация фичей
    'agg_primitives': ['count', 'max', 'min', 'mean', 'num_unique', 'percent_true', 'all', 'time_since_last'],  # добавлены примитивы для работы с датами
#     'agg_primitives': ['max'], # 'min'],
#     'trans_primitives': ['percentile'],
    'trans_primitives': ['percentile', 'not', 'cum_sum', 'cum_max', 'cum_mean', 'cum_min', 'month', 'day', 'weekday', 'week', 'time_since_previous'],  # добавлены примитивы для работы с датами
    'output_file_name': 'fm_many_tables_selected_features.csv',
    'file_name_for_features': 'fn_many_tables.json',  # имя файла для сохранения списка сгенерированных фич (используется для отбора фич после модуля selector)
    'generate_selected_features': True,  # указание на то, что нужно сгенерировать только выбранные фичи модулем selector
    'file_with_selected_features': '/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/short_df_sample.csv'  # результирующая таблица после модуля selector
}

In [4]:
%%time
# Запуск генератора для нескольких таблиц и всех фич
fg_many_tables_all_features = generator.FeatureGenerator(parameters_many_tables_all_features)
fg_many_tables_all_features()

Built 2462 features
EntitySet scattered to 2 workers in 205 seconds
Elapsed: 25:44 | Progress:  95%|█████████▌

distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing


Elapsed: 26:08 | Progress: 100%|██████████
CPU times: user 10min 36s, sys: 1min 44s, total: 12min 21s
Wall time: 38min 50s


In [4]:
%%time
# Запуск генератора для нескольких таблиц и отобранных фич
fg_many_tables_selected_features = generator.FeatureGenerator(parameters_many_tables_selected_features)
fg_many_tables_selected_features()

EntitySet scattered to 2 workers in 57 seconds
Elapsed: 05:42 | Progress: 100%|██████████
CPU times: user 3min 16s, sys: 19.7 s, total: 3min 35s
Wall time: 9min 57s


### Одна таблица

In [31]:
# Параметры для спецификации одной таблицы таблиц
parameters_one_table_all_features = {
    'path_data': '/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data',
    'tables': {
        'bureau_balance.csv': False,
    }, # в качестве значений указываются уникальные индексы для таблиц
    'main_table': 'bureau_balance.csv', # у main_table ОБЯЗАТЕЛЬНО должен быть уникальный идентификатор
    'target': 'STATUS',
    'depth': 2,
    'relations': [],
    'time_indecies': {
        'bureau_balance.csv': 'bureau_balance_date'
    },
    'time_variables': {
        'bureau_balance.csv': ['bureau_balance_date'],
    },  # после получения от Ани таблицы с типами, удалить этот словарь, в этих списках должны быть поля time_index'
    'sep': ',', # разделитель в csv файлах
    'chunk_size': 0.5, # количество строчек обрабатываемых за момент времени (если число от 0 до 1 - то это процент строчек)
    'max_features': 10**5, # максимальное количество генерируемых фич
    'n_jobs': 2, # количество потоков, в которых запускается генерация фичей
#     'agg_primitives': ['count', 'max', 'min', 'median', 'mean', 'num_unique', 'sum'],
    'agg_primitives': ['max'], # 'min'],
    'trans_primitives': ['year', 'day', 'weekday', 'month'], # 'cum_sum'],
    'output_file_name': 'fm_one_table.csv',
    'file_name_for_features': 'fn_one_tables.json',  # имя файла для сохранения списка сгенерированных фич (используется для отбора фич после модуля selector)
    'generate_selected_features': False,  # указание на то, что нужно сгенерировать только выбранные фичи модулем selector
    'file_with_selected_features': None  # результирующая таблица после модуля selector
}

parameters_one_table_selected_features = {
    'path_data': '/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data',
    'tables': {
        'bureau_balance.csv': False,
    }, # в качестве значений указываются уникальные индексы для таблиц
    'main_table': 'bureau_balance.csv', # у main_table ОБЯЗАТЕЛЬНО должен быть уникальный идентификатор
    'target': 'STATUS',
    'depth': 2,
    'relations': [],
    'time_indecies': {
        'bureau_balance.csv': 'bureau_balance_date'
    },
    'time_variables': {
        'bureau_balance.csv': ['bureau_balance_date'],
    },  # после получения от Ани таблицы с типами, удалить этот словарь, в этих списках должны быть поля time_index'
    'sep': ',', # разделитель в csv файлах
    'chunk_size': 0.5, # количество строчек обрабатываемых за момент времени (если число от 0 до 1 - то это процент строчек)
    'max_features': 10**5, # максимальное количество генерируемых фич
    'n_jobs': 2, # количество потоков, в которых запускается генерация фичей
#     'agg_primitives': ['count', 'max', 'min', 'median', 'mean', 'num_unique', 'sum'],
    'agg_primitives': ['max'], # 'min'],
    'trans_primitives': ['year', 'day', 'weekday', 'month'], # 'cum_sum'],
    'output_file_name': 'fm_one_table_selected_features.csv',
    'file_name_for_features': 'fn_one_tables.json',  # имя файла для сохранения списка сгенерированных фич (используется для отбора фич после модуля selector)
    'generate_selected_features': True,  # указание на то, что нужно сгенерировать только выбранные фичи модулем selector
    'file_with_selected_features': '/Users/antonbaranov/Documents/Python_examples/FeatureGenerator/data/short_df_sample_from_one_table.csv'  # результирующая таблица после модуля selector
}

In [25]:
%%time
# Запуск генератора для одной таблицы и всех фич
fg_one_table_all_features = generator.FeatureGenerator(parameters_one_table_all_features)
fg_one_table_all_features()

Built 6 features
EntitySet scattered to 2 workers in 11 seconds
Elapsed: 00:37 | Progress: 100%|██████████
CPU times: user 51.2 s, sys: 7.71 s, total: 58.9 s
Wall time: 1min 35s


In [32]:
%%time
# Запуск генератора для одной таблицы и отобранных фич
fg_one_table_selected_features = generator.FeatureGenerator(parameters_one_table_selected_features)
fg_one_table_selected_features()

EntitySet scattered to 2 workers in 11 seconds
Elapsed: 00:35 | Progress: 100%|██████████
CPU times: user 52 s, sys: 7.42 s, total: 59.4 s
Wall time: 1min 33s


In [37]:
# primitives = ft.list_primitives()
# print(primitives.shape)

# primitives[primitives['type'] == 'aggregation']

In [38]:
# primitives[primitives['type'] == 'transform']